<a href="https://colab.research.google.com/github/adriellisantos/postech-fase05/blob/main/Notebook_do_Machine_Learning_Datathon_Fase_05_Pos_Tech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregamento, Tratamento e Limpeza dos Dados

In [61]:
#Importação das bibliotecas que serão utilizadas durante o processo

import numpy as np
import pandas as pd
import datetime as dt
import warnings
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

In [62]:
# Lê o arquivo JSON "applicants.json" e carrega em um DataFrame do pandas. Indica que as chaves do JSON devem ser interpretadas como índices do DataFrame
df_applicants = pd.read_json('bases/applicants.json', orient='index')

# Normaliza as colunas que contém objetos/dicionários aninhados, transformando cada chave do dicionário em uma coluna separada.
df_infomacoes_basicas = pd.json_normalize(df_applicants['infos_basicas'])
df_informacoes_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
df_informacoes_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
df_formacao_idiomas = pd.json_normalize(df_applicants['formacao_e_idiomas'])
df_cargo = pd.json_normalize(df_applicants['cargo_atual'])

#Junta lado a lado todos os DataFrames criados acima em um único DataFrame com todas as informações dos candidatos
df_applicants = pd.concat([df_infomacoes_basicas, df_informacoes_pessoais, df_informacoes_profissionais, df_formacao_idiomas, df_cargo], axis=1)

In [63]:
# Realiza a leitura "vagas.json" e carrega em um DataFrame do pandas. Indica que as chaves do JSON devem ser interpretadas como índices do DataFrame
df_vagas = pd.read_json('bases/vagas.json', orient='index')

#Redefine o índice do DataFrame 'df_vagas', transformando o índice atual em uma coluna
df_vagas.reset_index(inplace=True)
df_vagas.rename(columns={"index": "codigo_vaga"}, inplace=True)

# Renomeia a nova coluna criada para "codigo_vaga", representando o código/identificador da vaga.
df_vagas_infomacoes_basicas = pd.json_normalize(df_vagas['informacoes_basicas'])
df_vagas_perfil = pd.json_normalize(df_vagas['perfil_vaga'])
df_vagas_beneficios = pd.json_normalize(df_vagas['beneficios'])

# Concatena  coluna 'codigo_vaga', os dados  de informações básicas, o perfil da vaga, os benefícios em um único DataFrame
df_vagas = pd.concat([df_vagas[['codigo_vaga']], df_vagas_infomacoes_basicas, df_vagas_perfil, df_vagas_beneficios], axis=1)

In [64]:
#Realiza a leitura arquivo JSON "prospects.json" e carrega em um DataFrame.
df_prospects = pd.read_json('bases/prospects.json', orient='index')

#Redefine o índice do DataFrame, transformando o índice original em uma coluna.
df_prospects.reset_index(inplace=True)
#Renomeia a coluna criada a partir do índice para "codigo_vaga",
df_prospects.rename(columns={"index": "codigo_vaga"}, inplace=True)

df_prospects = df_prospects.explode('prospects').reset_index(drop=True)
df_prospects_detalhado = pd.json_normalize(df_prospects['prospects'])

# Concatena os dados
df_prospects = pd.concat([df_prospects.drop(columns=['prospects']), df_prospects_detalhado], axis=1)

In [65]:
#Remove colunas duplicadas do DataFrame df_applicants,mantendo apenas a primeira ocorrência de cada coluna
df_applicants = df_applicants.loc[:, ~df_applicants.columns.duplicated(keep='first')]

In [66]:
#Retorna o Index contendo os nomes de todas as colunas de df_vagas
df_vagas.columns

Index(['codigo_vaga', 'data_requicisao', 'limite_esperado_para_contratacao',
       'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
       'empresa_divisao', 'requisitante', 'analista_responsavel',
       'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
       'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome',
       'telefone', 'data_inicial', 'data_final', 'nome_substituto', 'pais',
       'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho',
       'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'valor_compra_2'],
      dtype='object')

In [67]:
# Cria uma lista com os nomes das colunas que irá continuar no DataFrame de candidatos, cada item é o nome de uma coluna presente em df_applicants.
col_applicants = [
    "codigo_profissional","objetivo_profissional",
    "pcd", "area_atuacao",
    "conhecimentos_tecnicos","certificacoes","outras_certificacoes",
    "nivel_profissional", "nivel_academico", "nivel_ingles",
    "nivel_espanhol","outro_idioma","cursos","cargo_atual","local"
]

# Reatribui o DataFrame 'df_applicants' para conter somente as colunas listadas
df_applicants = df_applicants[col_applicants]

In [68]:
#Confirmando o tamanho do dataframe
df_applicants.shape

(42482, 15)

In [69]:
#Somando quantos dados nulos possui no dataframe
df_applicants.isnull().sum()

,0
codigo_profissional,0
objetivo_profissional,0
pcd,0
area_atuacao,0
conhecimentos_tecnicos,0
certificacoes,0
outras_certificacoes,0
nivel_profissional,0
nivel_academico,0
nivel_ingles,0


In [70]:
#Verificando em cada coluna do df quantos células tem uma string vazia
(df_applicants == "").sum()

,0
codigo_profissional,0
objetivo_profissional,14416
pcd,36161
area_atuacao,35731
conhecimentos_tecnicos,39129
certificacoes,41954
outras_certificacoes,41583
nivel_profissional,42402
nivel_academico,34388
nivel_ingles,35638


In [71]:
#Criando uma série com a porcentagem de células vazias ("") em cada coluna do DataFrame df_applicants.
porcentagem_vazios = (df_applicants == "").sum() / df_applicants.shape[0]
print(porcentagem_vazios.round(2))

codigo_profissional       0.00
objetivo_profissional     0.34
pcd                       0.85
area_atuacao              0.84
conhecimentos_tecnicos    0.92
certificacoes             0.99
outras_certificacoes      0.98
nivel_profissional        1.00
nivel_academico           0.81
nivel_ingles              0.84
nivel_espanhol            0.84
outro_idioma              0.00
cursos                    0.03
cargo_atual               0.01
local                     0.60
dtype: float64


# Engenharia de Features

In [72]:
#Definindo uma função que filtra os valores indesejados de uma coluna específica de um DataFrame
def filtrar_caracteres(df, col):
    def limpar_objetivo(obj):
        #Definindo uma função que filtra/limpa valores indesejados de uma coluna específica
        obj = str(obj).strip()

        if pd.isna(obj) or obj in ["", ".", "-", " "]:
            return ""

        if re.match(r"^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$", obj):
            return ""

        if pd.to_datetime(obj, errors='coerce') is not pd.NaT:
            return ""

        if not re.search(r"[a-zA-Z]", obj):
            return ""

        obj = obj.replace('nan', '')

        #Removendo ocorrências da string nulas dentro do valor
        return obj

    #Aplicando a função de limpeza em todos os valores da coluna
    df[col] = df[col].apply(limpar_objetivo)

    return df

In [73]:
#Lista de colunas do DataFrame df_applicants que serão limpas
col_limpar = [
    "objetivo_profissional", "pcd", "area_atuacao", "conhecimentos_tecnicos",
    "certificacoes", "outras_certificacoes", "nivel_profissional", "nivel_academico",
    "nivel_ingles", "nivel_espanhol", "outro_idioma", "cursos", "cargo_atual", "local"
]

#Para cada coluna da lista col_limpar, aplica a função filtrar_caracteres que limpa valores indesejados
for col in col_limpar:
    df_applicants = filtrar_caracteres(df_applicants, col)


In [74]:
#Filtrando o df df_applicants, mantendo apenas as linhas em que a coluna 'objetivo_profissional' **não está vazia**
df_applicants = df_applicants[df_applicants['objetivo_profissional'] != '']

In [75]:
#Confirmando o tamanho do df_applicants
df_applicants.shape

(27994, 15)

In [76]:
##Verificando em cada coluna do df quantos células tem uma string vazia
(df_applicants == "").sum()

,0
codigo_profissional,0
objetivo_profissional,0
pcd,22150
area_atuacao,21761
conhecimentos_tecnicos,24777
certificacoes,27508
outras_certificacoes,27149
nivel_profissional,27931
nivel_academico,20455
nivel_ingles,21734


In [77]:
#Função que concatenadas  grupos de colunas de um df, para cada grupo de colunas, ela gera uma coluna única que combina os valores não vazios, prefixando o nome da coluna.
def criar_grupos_concac(df, id_coluna, grupos_colunas):
    #Dicionário para armazenar os DataFrames resultantes de cada grupo
    dfs_resultado = {}

    for nome_grupo, colunas in grupos_colunas.items():
        df_temp = df[[id_coluna] + colunas].copy()

        #Função interna que concatena os valores de uma linha em uma string única
        def concatenar_linha(row):
            textos_validos = [
                f"{coluna}: {str(row[coluna]).strip()}"
                for coluna in colunas
                if str(row[coluna]).strip() != ""
            ]
            return ' | '.join(textos_validos)

        #Aplicando a função linha a linha, criando a coluna concatenada para o grupo
        df_temp[f"perfil_concatenado_{nome_grupo}"] = df_temp.apply(concatenar_linha, axis=1)

        #Removendo linhas que ficaram vazias após concatenar
        df_temp = df_temp[df_temp[f"perfil_concatenado_{nome_grupo}"] != ""]

        #Adicionando ao dicionário de resultados apenas a coluna de ID e a coluna concatenada
        dfs_resultado[nome_grupo] = df_temp[[id_coluna, f"perfil_concatenado_{nome_grupo}"]]

    return dfs_resultado


In [78]:
#Criando lista de colunas relacionadas a habilidades técnicas e experiência profissional
hard_skills = [
    "objetivo_profissional",
    "area_atuacao",
    "conhecimentos_tecnicos",
    "nivel_profissional",
    "cargo_atual",
    "cursos"
]

#Criando lista de colunas relacionadas à formação acadêmica e proficiência em idiomas
education = [
    "certificacoes",
    "outras_certificacoes",
    "nivel_academico",
    "nivel_ingles",
    "nivel_espanhol",
    "outro_idioma"
]

#Criando lista de colunas relacionadas a informações pessoais
personal = [
    "pcd",
    "local"
]

#Dicionário que agrupa as listas
grupos_col = {
    "hard_skills": hard_skills,
    "education": education,
    "personal": personal
}

In [79]:
#Chamando a função 'criar_grupos_concac' para gerar colunas concatenadas para cada grupo de colunas definido em 'grupos_col'
dfs_por_grupo = criar_grupos_concac(df_applicants, "codigo_profissional", grupos_col)

In [80]:
#Extraindo os dfs individuais do dicionário de grupos concatenados cada df
df_hard_skills = dfs_por_grupo["hard_skills"]
df_education = dfs_por_grupo["education"]
df_personal = dfs_por_grupo["personal"]

#Faz o merge dos dfs para criar um DataFrame único
df_unico = df_hard_skills.merge(df_education, on="codigo_profissional", how="outer")
df_unico = df_unico.merge(df_personal, on="codigo_profissional", how="outer")

In [81]:
#Conta para cada coluna do df df_unico quantas células contêm uma string vazia
(df_unico == "").sum()

,0
codigo_profissional,0
perfil_concatenado_hard_skills,0
perfil_concatenado_education,0
perfil_concatenado_personal,0


In [82]:
#Verificando o tipo de dados do df
df_unico.dtypes

,0
codigo_profissional,object
perfil_concatenado_hard_skills,object
perfil_concatenado_education,object
perfil_concatenado_personal,object


In [83]:
#Convertendo os valores da coluna 'codigo_profissional' para o tipo inteiro
df_unico['codigo_profissional'] = df_unico['codigo_profissional'].astype(int)

In [84]:
#Retornando os nomes de todas as colunas do df_prospects
df_prospects.columns

Index(['codigo_vaga', 'titulo', 'modalidade', 'nome', 'codigo',
       'situacao_candidado', 'data_candidatura', 'ultima_atualizacao',
       'comentario', 'recrutador'],
      dtype='object')

In [85]:
#Verifica o tamanho do df
df_prospects.shape

(56702, 10)

In [86]:
#Soma os valores nulos do df
df_prospects.isnull().sum()

,0
codigo_vaga,0
titulo,0
modalidade,0
nome,2943
codigo,2943
situacao_candidado,2943
data_candidatura,2943
ultima_atualizacao,2943
comentario,2943
recrutador,2943


In [87]:
#Conta para cada coluna do df_prospects quantas células contem uma string vazia
(df_prospects == "").sum()

,0
codigo_vaga,0
titulo,2943
modalidade,55019
nome,0
codigo,0
situacao_candidado,0
data_candidatura,0
ultima_atualizacao,3913
comentario,39201
recrutador,0


In [88]:
#Selecionando apenas um subconjunto específico de colunas do df_prospects
df_prospects = df_prospects[["codigo","codigo_vaga","titulo","situacao_candidado"]]

In [89]:
#Contando para cada coluna do df_prospects, quantos valores ausentes existem
df_prospects.isnull().sum()

,0
codigo,2943
codigo_vaga,0
titulo,0
situacao_candidado,2943


In [90]:
#Removendo todas as linhas do  df_prospects que contêm algum valor ausente
df_prospects.dropna(inplace=True)

In [91]:
#Verifica o tamanho do df
df_prospects.shape

(53759, 4)

In [92]:
#Conta para cada coluna do df_prospects quantas células contem uma string vazia
(df_prospects == "").sum()

,0
codigo,0
codigo_vaga,0
titulo,0
situacao_candidado,0


In [93]:
#Retornando os valores únicos presentes na coluna 'situacao_candidado' do df_prospects
df_prospects['situacao_candidado'].unique()

array(['Encaminhado ao Requisitante', 'Contratado pela Decision',
       'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente',
       'Prospect', 'Não Aprovado pelo RH', 'Aprovado',
       'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Contratado como Hunting',
       'Desistiu da Contratação', 'Entrevista com Cliente',
       'Documentação CLT', 'Recusado', 'Documentação Cooperado',
       'Sem interesse nesta vaga', 'Encaminhar Proposta',
       'Proposta Aceita'], dtype=object)

In [94]:
#Definindo listas de status considerados aprovados e não aprovados
status_aprov = [
    'Aprovado',
    'Contratado pela Decision',
    'Contratado como Hunting',
    'Proposta Aceita',
    'Encaminhar Proposta'
]

status_neg = [
    'Não Aprovado pelo Cliente',
    'Não Aprovado pelo RH',
    'Não Aprovado pelo Requisitante',
    'Desistiu',
    'Desistiu da Contratação',
    'Recusado',
    'Sem interesse nesta vaga'
]

#Removendo espaços em branco antes e depois dos valores na coluna 'situacao_candidado'
df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].str.strip()

#Convertendo os status da coluna para valores binários:
df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].apply(
    lambda x: 1 if x in status_aprov else 0
)


In [95]:
#Contando a frequência de cada valor único na coluna 'situacao_candidado' do df_prospects
df_prospects['situacao_candidado'].value_counts()

,count
situacao_candidado,
0,50563
1,3196


In [96]:
#Selecionando apenas as 'codigo' e 'situacao_candidado' do df_prospects e cria um novo chamado df_prospects_concac
df_prospects_concac = df_prospects[['codigo','situacao_candidado']]

In [97]:
#Verifica o tipo das colunas do df
df_prospects_concac.dtypes

,0
codigo,object
situacao_candidado,int64


In [98]:
# Renomendo a coluna 'codigo' para 'codigo_profissional' no DataFrame df_prospects_concac
df_prospects_concac.rename(columns={'codigo': 'codigo_profissional'}, inplace=True)

#Convertendo os valores da coluna 'codigo_profissional' para tipo inteiro (int)
df_prospects_concac['codigo_profissional'] = df_prospects_concac['codigo_profissional'].astype(int)

In [99]:
#Selecionando os códigos dos profissionais que foram aprovados e armazena na variável deletar_aprov
deletar_aprov = df_prospects_concac[df_prospects_concac['situacao_candidado'] == 1]['codigo_profissional']

In [100]:
#Criando o df_prospects_tratado contendo apenas os profissionais *não aprovados* e contabiliza quantos registros existem para cada profissional
df_prospects_tratado = df_prospects_concac[df_prospects_concac['situacao_candidado'] == 0].groupby(by='codigo_profissional').count().reset_index()

In [101]:
#Renomendo a coluna 'situacao_candidado' para 'quantidade_prospect' no  df_prospects_tratado
df_prospects_tratado.rename(columns={'situacao_candidado':'quantidade_prospect'},inplace=True)

In [102]:
#Realiza o merge entre dois dfs
df_merge = pd.merge(df_unico, df_prospects_tratado, on=['codigo_profissional'], how='left')

In [103]:
#Removendo do df_merge todas as linhas cujo 'codigo_profissional esteja na lista deletar_aprov.
df_merge = df_merge[~df_merge['codigo_profissional'].isin(deletar_aprov)]

In [104]:
#Substituindo os valores ausentes na coluna 'quantidade_prospect' por 0
df_merge['quantidade_prospect'].fillna(0,inplace=True)

In [105]:
#Convertendo os valores da coluna 'quantidade_prospect' para o tipo int
df_merge['quantidade_prospect'] = df_merge['quantidade_prospect'].astype(int)

In [106]:
#Retornando a quantidade de valores únicos em cada coluna do df_merge
df_merge.nunique()

,0
codigo_profissional,27011
perfil_concatenado_hard_skills,10807
perfil_concatenado_education,1800
perfil_concatenado_personal,996
quantidade_prospect,35


In [107]:
#Criando uma cópia do  df_merge contendo apenas as colunas relacionadas

df_hard_skills = df_merge[["codigo_profissional", "perfil_concatenado_hard_skills", "quantidade_prospect"]].copy()

df_education = df_merge[["codigo_profissional", "perfil_concatenado_education"]].copy()

df_personal = df_merge[["codigo_profissional", "perfil_concatenado_personal"]].copy()

In [108]:
#Retornando os nomes de todas as colunas do df_vagas
df_vagas.columns

Index(['codigo_vaga', 'data_requicisao', 'limite_esperado_para_contratacao',
       'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
       'empresa_divisao', 'requisitante', 'analista_responsavel',
       'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
       'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome',
       'telefone', 'data_inicial', 'data_final', 'nome_substituto', 'pais',
       'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho',
       'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'valor_compra_2'],
      dtype='object')

In [109]:
#Definindo uma lista de colunas relevantes para análise das vagas
col_vagas = [
    'codigo_vaga',
    'titulo_vaga',
    'vaga_especifica_para_pcd',
    'nivel profissional',
    'nivel_academico',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'areas_atuacao',
    'principais_atividades',
    'competencia_tecnicas_e_comportamentais',
    'estado',
    'cidade'
]

#Selecionando apenas as colunas definidas em col_vagas no DataFrame df_vagas
df_vagas = df_vagas[col_vagas]

In [110]:
df_vagas['titulo_vaga'] = df_vagas['titulo_vaga'].str.replace(r'\b\w*\d+\s*-\s*', '', regex=True).str.strip()
df_vagas['titulo_vaga'] = df_vagas['titulo_vaga'].str.replace(r'\s*-\s*\d+$', '', regex=True).str.strip()

In [111]:
#Removendo padrões numéricos específicos do início do título da vaga
df_vagas['principais_atividades'] = (
    df_vagas['principais_atividades']
    .str.replace(r'\n+', ' ', regex=True)
    .str.replace('•', '')
    .str.replace('', '')
    .str.strip()
)


In [112]:
#Limpando e padronizando o texto da coluna 'competencia_tecnicas_e_comportamentais' no df_vagas
df_vagas['competencia_tecnicas_e_comportamentais'] = (
    df_vagas['competencia_tecnicas_e_comportamentais']
    .str.replace(r'\n+', ' ', regex=True)
    .str.replace('•', '')
    .str.replace('', '')
    .str.strip()
)

In [113]:
#Contando a quantidade de valores ausentes em cada coluna do df_vagas
df_vagas.isnull().sum()

,0
codigo_vaga,0
titulo_vaga,0
vaga_especifica_para_pcd,0
nivel profissional,0
nivel_academico,0
nivel_ingles,0
nivel_espanhol,0
outro_idioma,0
areas_atuacao,0
principais_atividades,0


In [114]:
#Contando para cada coluna do df_vagas quantas células contêm uma string vazia
(df_vagas == "").sum()

,0
codigo_vaga,0
titulo_vaga,0
vaga_especifica_para_pcd,1948
nivel profissional,0
nivel_academico,0
nivel_ingles,0
nivel_espanhol,1304
outro_idioma,13708
areas_atuacao,15
principais_atividades,3


In [115]:
#Renomendo a coluna 'nivel profissional' para 'nivel_profissional' no df_vagas

df_vagas.rename(columns={"nivel profissional":"nivel_profissional"},inplace=True)

In [116]:
#Definindo listas de colunas de interesse para cada grupo de informações sobre as vagas
hard_skills_vagas = [
    "titulo_vaga"
    ,"areas_atuacao"
    ,"principais_atividades"
    ,"competencia_tecnicas_e_comportamentais"
    ,"nivel_profissional"
]

education_vagas = [
    "nivel_academico"
    ,"nivel_ingles"
    ,"nivel_espanhol"
    ,"outro_idioma"
]

personal_vagas = [
    "vaga_especifica_para_pcd"
    ,"estado"
    ,"cidade"
]

grupos_colunas_vagas = {
    "hard_skills_vagas": hard_skills_vagas,
    "education_vagas": education_vagas,
    "personal_vagas": personal_vagas
}

#Aplicando a função criar_grupos_concac para concatenar as informações de cada grupo em uma coluna
dfs_por_grupo_vaga = criar_grupos_concac(df_vagas, "codigo_vaga", grupos_colunas_vagas)

#Extraindo os DataFrames de cada grupo do dicionário resultante
df_hard_skills_vagas = dfs_por_grupo_vaga["hard_skills_vagas"]
df_education_vagas = dfs_por_grupo_vaga["education_vagas"]
df_personal_vagas = dfs_por_grupo_vaga["personal_vagas"]

In [117]:
#Removendo linhas do df_personal onde o perfil concatenado está vazio
df_personal = df_personal[df_personal['perfil_concatenado_personal'] != '']
df_education = df_education[df_education['perfil_concatenado_education'] != '']

# Seleção de NLP e Embeddings

In [118]:
from sentence_transformers import SentenceTransformer
import pandas as pd

#Carregando um modelo pré-treinado para gerar embeddings de frases em múltiplos idiomas
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('all-MiniLM-L12-v22')

#Função para gerar embeddings de uma coluna de texto em um df
def gera_embeddings(df, col_texto, prefixo_nome_embedding):
    df = df.copy()
    df[col_texto] = df[col_texto].astype(str)

    embeddings = model.encode(df[col_texto].tolist(), show_progress_bar=True)
    embeddings_df = pd.DataFrame(
        embeddings,
        columns=[f'{prefixo_nome_embedding}_embedding_{i}' for i in range(embeddings.shape[1])]
    )

    #Concatena os embeddings ao df original e retorna o resultado
    return pd.concat([df.reset_index(drop=True), embeddings_df], axis=1)

In [119]:
#Gera embeddings para o texto concatenado de cada grupo de informações dos candidatos
df_hard_skills_embed = gera_embeddings(df_hard_skills, 'perfil_concatenado_hard_skills', 'hard')
df_education_embed = gera_embeddings(df_education, 'perfil_concatenado_education', 'edu')
df_personal_embed = gera_embeddings(df_personal, 'perfil_concatenado_personal', 'pers')

Batches:   0%|          | 0/845 [00:00<?, ?it/s]

Batches:   0%|          | 0/845 [00:00<?, ?it/s]

Batches:   0%|          | 0/845 [00:00<?, ?it/s]

In [120]:
#Definindo os pesos de cada grupo de informações para um cálculo de similaridade
pesos = {
    'hard_skills': 0.5,
    'education': 0.3,
    'personal': 0.2
}

In [121]:
def aplica_peso(df_embed, peso, prefixo):
    #Criando uma lista com o nome das colunas de df_embed que começam com o prefixo indicado
    cols_embed = [col for col in df_embed.columns if col.startswith(prefixo)]

    #Multiplicando todos os valores dessas colunas pelo peso informado.
    df_embed[cols_embed] = df_embed[cols_embed] * peso
    return df_embed

In [122]:
#Aplica o peso definido para o grupo aos embeddings
df_hard_skills_embed = aplica_peso(df_hard_skills_embed, pesos['hard_skills'], 'hard')
df_educacao_embed = aplica_peso(df_education_embed, pesos['education'], 'edu')
df_pessoal_embed = aplica_peso(df_personal_embed, pesos['personal'], 'pers')

In [123]:
#Faz um merge entre os dfs de embeddings de hard skills e de educacao, usando a coluna 'codigo_profissional' como chave
df_temp = df_hard_skills_embed.merge(
    df_education_embed, on='codigo_profissional', how='left', suffixes=('', '_edu')
)

#Faz um merge entre o df_temp e o df de embeddings de df_personal_embed
df_final = df_temp.merge(
    df_personal_embed, on='codigo_profissional', how='left', suffixes=('', '_pers')
)

In [124]:
#Criando um novo df a partir de df_final, selecionando apenas as colunas cujo nome NÃO contém a expressão perfil_concatenado
df_final_candidatos = df_final[df_final.columns[~df_final.columns.str.contains("perfil_concatenado", case=False)]]

In [125]:
#Substituindo todos os valores ausente no df_final_candidatos por 0
df_final_candidatos.fillna(0,inplace=True)

In [126]:
#Gerando embeddings para os textos
df_hard_skills_embed_vaga = gera_embeddings(df_hard_skills_vagas, 'perfil_concatenado_hard_skills_vagas', 'vhard')
df_education_embed_vaga = gera_embeddings(df_education_vagas, 'perfil_concatenado_education_vagas', 'vedu')
df_personal_embed_vaga = gera_embeddings(df_personal_vagas, 'perfil_concatenado_personal_vagas', 'vpers')

Batches:   0%|          | 0/441 [00:00<?, ?it/s]

Batches:   0%|          | 0/441 [00:00<?, ?it/s]

Batches:   0%|          | 0/441 [00:00<?, ?it/s]

In [127]:
#Aplicando o peso definido para "hard_skills" (0.5) em todos os vetores
df_hard_skills_vaga_embed = aplica_peso(df_hard_skills_embed_vaga, pesos['hard_skills'], 'vhard')
df_education_vaga_embed = aplica_peso(df_education_embed_vaga, pesos['education'], 'vedu')
df_personal_vaga_embed = aplica_peso(df_personal_embed_vaga, pesos['personal'], 'vpers')

In [128]:
# Une os embeddings de hard skills df_hard_skills_vaga_embed com os embeddings de education de df_education_vaga_embed, usando a coluna 'codigo_vaga' como chave

df_vagas_embed = df_hard_skills_vaga_embed.merge(
    df_education_vaga_embed, on='codigo_vaga', how='left'
).merge(
    #Faz outro merge para juntar também os embeddings do grupo df_personal_vaga_embed, novamente usando 'codigo_vaga' como chave e garantindo que nenhuma vaga do conjunto base hard skills seja perdida
    df_personal_vaga_embed, on='codigo_vaga', how='left'
)

df_vagas_embed.fillna(0, inplace=True)


In [129]:
#Criando o df_final_vagas contendo apenas as colunas de embeddings e outras colunas relevantes, removendo todas as colunas cujo nome contém "perfil_concatenado"
df_final_vagas = df_vagas_embed[df_vagas_embed.columns[~df_vagas_embed.columns.str.contains("perfil_concatenado", case=False)]]

# Salvamento do Modelo

In [130]:
import pickle
import gzip

df_final_candidatos.to_pickle("df_finalcandidatos.pkl")
df_final_vagas.to_pickle("df_finalvagas.pkl")
df_applicants.to_pickle("df_applicants.pkl")
df_vagas.to_pickle("df_vagas.pkl")


# Validação do Modelo e Treinamento

In [131]:
#Juntar as features de candidatos e vagas para cada par registrado em prospects
df_pairs = (
    df_prospects[['codigo_vaga','codigo','situacao_candidado']]
      .rename(columns={'codigo':'codigo_profissional'})
      .astype({'codigo_profissional': int}) # Convert to int before merging
      .merge(df_final_candidatos, on='codigo_profissional', how='left')
      .merge(df_final_vagas, on='codigo_vaga', how='left')
)

# Remove colunas não numéricas (ex.: nomes, textos) se houver
X = df_pairs[[col for col in df_pairs.columns if 'embedding' in col]]
y = df_pairs['situacao_candidado']

In [132]:
from sklearn.model_selection import train_test_split

#Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [133]:
#Criando uma lista com os nomes das colunas que contêm a palavra 'embedding'
embedding_cols_candidatos = [c for c in df_final_candidatos.columns if 'embedding' in c]


In [134]:
#Selecionando apenas as colunas de embeddings dos candidatos no DataFrame
candidatos_vectors = df_final_candidatos[embedding_cols_candidatos].values


In [135]:
embedding_cols_vaga = [c for c in df_final_vagas.columns if 'embedding' in c]
codigo_vaga = 5900
vaga_vector = df_final_vagas[df_final_vagas['codigo_vaga']==codigo_vaga][embedding_cols_vaga].values

if vaga_vector.ndim == 1:
    vaga_vector = vaga_vector.reshape(1, -1)

In [136]:
from sklearn.metrics.pairwise import cosine_similarity

sim_scores = cosine_similarity(candidatos_vectors, vaga_vector).flatten()


In [137]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

#Seleciona todas as colunas de df_final_candidatos cujo nome possui 'embedding'
embedding_cols_candidatos = [c for c in df_final_candidatos.columns if 'embedding' in c]

#Extrai em forma de array os vetores de embeddings de todos os candidatos
candidatos_vectors = df_final_candidatos[embedding_cols_candidatos].values

#Define o código da vaga a ser analisada
codigo_vaga_input = 101

#Seleciona as colunas de embeddings do df de vagas
embedding_cols_vaga = [c for c in df_final_vagas.columns if 'embedding' in c]

#Obtém o vetor de embeddings referente à vaga com o código especificado
vaga_vector = df_final_vagas[
    df_final_vagas['codigo_vaga'] == codigo_vaga_input
][embedding_cols_vaga].values

# Garante que vaga_vector seja 2D
if vaga_vector.ndim == 1:
    vaga_vector = vaga_vector.reshape(1, -1)

# Calcula a similaridade de cosseno entre cada vetor de candidato e o vetor da vaga selecionada
sim_scores = cosine_similarity(candidatos_vectors, vaga_vector).flatten()

#Filtra o df de prospects para obter apenas os registros da vaga escolhida
df_y_true = df_prospects[df_prospects['codigo_vaga'] == codigo_vaga_input]

#Faz um merge para associar a cada candidato sua situação na vaga
df_merge = df_final_candidatos.merge(
    df_y_true[['codigo', 'situacao_candidado']]
          .rename(columns={'codigo': 'codigo_profissional'})
          .astype({'codigo_profissional': int}),
    on='codigo_profissional',
    how='left'
)

# Cria o vetor de rótulos reais se um candidato não tiver registro em df_prospects atribui 0
y_true = df_merge['situacao_candidado'].fillna(0).values

#Define um limiar para considerar um candidato como "aprovado" pela similaridade.
threshold = 0.7

#Gera as predições binárias:
y_pred = (sim_scores >= threshold).astype(int)

#Calcula e exibe a acurácia
print("Accuracy:", accuracy_score(y_true, y_pred))


Accuracy: 0.9929658287364407


In [138]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendar_candidatos_cosine(codigo_vaga_input, df_candidatos_final, df_vagas_final, top_n=5):
    #Filtrando a vaga pelo código informado
    vaga_embed = df_vagas_final[df_vagas_final['codigo_vaga'] == codigo_vaga_input]
    if vaga_embed.empty:
        return "Vaga não encontrada."

    #Selecionando colunas de embeddings para vaga e candidatos
    embedding_cols_vaga = [col for col in df_vagas_final.columns if 'embedding' in col]
    embedding_cols_candidatos = [col for col in df_candidatos_final.columns if 'embedding' in col]

    #Extraindo vetor embedding da vaga (transforma para 2D se necessário)
    vaga_vector = vaga_embed[embedding_cols_vaga].values
    if vaga_vector.ndim == 1:
        vaga_vector = vaga_vector.reshape(1, -1)

    #Extraindo vetores embeddings dos candidatos
    candidatos_vectors = df_candidatos_final[embedding_cols_candidatos].values

    #Calcula similaridade do cosseno entre candidatos e a vaga
    sim_scores = cosine_similarity(candidatos_vectors, vaga_vector).flatten()

    #Criando cópia para adicionar as pontuações
    candidatos_disponiveis = df_candidatos_final.copy()
    candidatos_disponiveis['score_cosine'] = sim_scores

    #Score em percentual formatado
    candidatos_disponiveis['score_cosine_percentual'] = (candidatos_disponiveis['score_cosine'] * 100).round(2).astype(str) + '%'

    #Garante tipo correto para merge
    df_applicants['codigo_profissional'] = df_applicants['codigo_profissional'].astype(int)
    candidatos_disponiveis['codigo_profissional'] = candidatos_disponiveis['codigo_profissional'].astype(int)

    # Merge para adicionar informações extras dos candidatos
    candidatos_com_info = candidatos_disponiveis.merge(
        df_applicants[['codigo_profissional', 'objetivo_profissional']],
        on='codigo_profissional',
        how='left'
    )

    # Retorna top N candidatos ordenados pela pontuação
    return candidatos_com_info.sort_values('score_cosine', ascending=False).head(top_n)[
        ['codigo_profissional', 'objetivo_profissional', 'score_cosine_percentual', 'quantidade_prospect']
    ]


In [139]:
#Testando a função de recomendação
codigo_vaga = 5900

resultado = recomendar_candidatos_cosine(codigo_vaga, df_final_candidatos, df_final_vagas, top_n=5)

resultado

,codigo_profissional,objetivo_profissional,score_cosine_percentual,quantidade_prospect
4909,16865,DevOps,81.57%,1
7114,23565,Atuar com a cultura DevOps,81.03%,2
7044,23398,ANALISTA DESENVOLVEDOR DE SISTEMAS,79.19%,0
9094,28744,"Analista de Infraestrutura, Administrador de R...",77.23%,2
8931,28283,Analista de Suporte,77.2%,1
